In [1]:
import json
import os

import pandas as pd

In [2]:
run_dir = "/share/kuleshov/yzs2/runs/dllm-dev/"

In [3]:
runs = {
    # "AR 28": {
    #     "Model": "AR",
    #     "Layers": "28",
    #     "path": "gsm8k_lr1e-5_bsz1_warm10ba_alphaf0.5_max-dur30000ba_amp_bf16_layers28_ar_FT2B"
    # },
    # "MDLM 28": {
    #     "Model": "MDLM",
    #     "Layers": "28",
    #     "path": "gsm8k_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_layers-1_mdlm_2BFT"
    # },
    "BD3LM 17": {
        "Model": "BD3LM",
        "Layers": "17",
        "path": "gsm8k_block4_evalblock4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_layers17_bd3lm_2B-FT",
    },
    "E2D2_Enc28_Dec14": {
        "Model": "E2D2",
        "Layers": "28 enc / 14 dec",
        "path": "gsm8k_block4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec14_e2d2_2BFT_tie-weights",
    },
    "BD3LM 21": {
        "Model": "BD3LM",
        "Layers": "21",
        "path": "gsm8k_block4_evalblock4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_layers21_bd3lm_2BFT",
    },
    "E2D2_Enc28_Dec21": {
        "Model": "E2D2",
        "Layers": "28 enc / 21 dec",
        "path": "gsm8k_block4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec21_e2d2_2B-FT_kv_tie-weights",
    },
    "BD3LM 28": {
        "Model": "BD3LM",
        "Layers": "28",
        "path": "gsm8k_block4_evalblock4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_layers28_bd3lm_2BFT",
    },
    "E2D2_Enc28_Dec24": {
        "Model": "E2D2",
        "Layers": "28 enc / 24 dec",
        "Block size": 4,
        "path": "gsm8k_block4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec24_e2d2_2B-FT_kv_tie-weights",
    },
    # "E2D2_Enc28_Dec14 BS2": {
    #     "Model": "E2D2",
    #     "Layers": "28 enc / 14 dec",
    #     "Block size": 2,
    #     "path": "gsm8k_block2_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec14_e2d2_2B-FT_share_kv_tie-weights",
    # },
    # "E2D2_Enc28_Dec14 BS4": {
    #     "Model": "E2D2",
    #     "Layers": "28 enc / 14 dec",
    #     "Block size": 4,
    #     "path": "gsm8k_block4_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec14_e2d2_2BFT_tie-weights",
    # },
    # "E2D2_Enc28_Dec14 BS8": {
    #     "Model": "E2D2",
    #     "Layers": "28 enc / 14 dec",
    #     "Block size": 8,
    #     "path": "gsm8k_block8_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec14_e2d2_2B-FT_share_kv_tie-weights",
    # },
    # "E2D2_Enc28_Dec14 BS16": {
    #     "Model": "E2D2",
    #     "Layers": "28 enc / 14 dec",
    #     "Block size": 16,
    #     "path": "gsm8k_block16_lr1e-5_bsz1_warm100ba_alphaf0.5_max-dur30000ba_amp_bf16_enc28_TOPdec14_e2d2_2B-FT_share_kv_tie-weights",
    # },
}

In [11]:
runs_by_T = {}
for k, v in runs.items():
    lm_eval_output_path = os.path.join(run_dir, v["path"], "lm_eval_harness_output")
    for T in [1, 2, 4]:
        for align_inputs in ["true"]:  # , "false"]:
            runs_by_T[f"{k}_{T=}_{align_inputs=}"] = {kk: vv for kk, vv in v.items()}
            runs_by_T[f"{k}_{T=}_{align_inputs=}"]["T"] = T
            runs_by_T[f"{k}_{T=}_{align_inputs=}"]["align"] = align_inputs
            T_lm_eval_output_path = os.path.join(
                lm_eval_output_path,
                # f"L256_block_size{T}-do_samplefalse-sampling_strategypredict_and_noise-T{T}_first_hittingtrue-confidence_based_noisingtrue-confidence_margin_based_noisingfalse-align_inputs_to_blocks{align_inputs}"
                f"L256_block_size4-do_samplefalse-sampling_strategypredict_and_noise-T{T}_first_hittingtrue-confidence_based_noisingtrue-confidence_margin_based_noisingfalse-align_inputs_to_blocks{align_inputs}",
            )
            try:
                with open(
                    os.path.join(T_lm_eval_output_path, "throughput-rank0.json"), "r"
                ) as f:
                    tput = json.load(f)
                runs_by_T[f"{k}_{T=}_{align_inputs=}"][
                    r"Inference Throughput (Tok/sec $\uparrow$)"
                ] = rf"{tput['throughput_mean']:0.2f} $\pm$ {tput['throughput_std']:0.2f}"
            except FileNotFoundError as e:
                # print(e)
                runs_by_T[f"{k}_{T=}_{align_inputs=}"][
                    r"Inference Throughput (Tok/sec $\uparrow$)"
                ] = "N/A"
            try:
                with open(os.path.join(T_lm_eval_output_path, "metrics.txt"), "r") as f:
                    # print(k, f"{T=}")
                    # print(f"Throughput: {tput['throughput_mean']:0.2f} ± {tput['throughput_std']:0.2f}")
                    lines = f.readlines()
                    #             print("".join(lines))
                    for line in lines:
                        if "flexible-extract" in line:
                            runs_by_T[f"{k}_{T=}_{align_inputs=}"][
                                r"0-shot pass @1 ($\uparrow$)"
                            ] = float(line.split("|±")[0].split("|")[-1]) * 100
            except FileNotFoundError as e:
                # print(e)
                runs_by_T[f"{k}_{T=}_{align_inputs=}"][
                    r"0-shot pass @1 ($\uparrow$)"
                ] = -1.0

In [12]:
df = pd.DataFrame.from_dict(runs_by_T, orient="index").reset_index()

In [14]:
pd.options.display.float_format = "{:,.2f}".format
cols = ["Model", "Layers", "T", r"0-shot pass @1 ($\uparrow$)"]
filt_df = df
# [
# (df[r"Inference Throughput (Tok/sec $\uparrow$)"] != "N/A")
# &
# (df["T"] == 4)
# ]
display(filt_df[cols])
print(filt_df[cols].to_markdown(index=False))

,Model,Layers,T,0-shot pass @1 ($\uparrow$)
0,BD3LM,17,1,4.09
1,BD3LM,17,2,11.98
2,BD3LM,17,4,15.39
3,E2D2,28 enc / 14 dec,1,14.03
4,E2D2,28 enc / 14 dec,2,30.17
5,E2D2,28 enc / 14 dec,4,40.33
6,BD3LM,21,1,7.73
7,BD3LM,21,2,21.08
8,BD3LM,21,4,29.42
9,E2D2,28 enc / 21 dec,1,16.68


| Model   | Layers          |   T |   0-shot pass @1 ($\uparrow$) |
|:--------|:----------------|----:|------------------------------:|
| BD3LM   | 17              |   1 |                          4.09 |
| BD3LM   | 17              |   2 |                         11.98 |
| BD3LM   | 17              |   4 |                         15.39 |
| E2D2    | 28 enc / 14 dec |   1 |                         14.03 |
| E2D2    | 28 enc / 14 dec |   2 |                         30.17 |
| E2D2    | 28 enc / 14 dec |   4 |                         40.33 |
| BD3LM   | 21              |   1 |                          7.73 |
| BD3LM   | 21              |   2 |                         21.08 |
| BD3LM   | 21              |   4 |                         29.42 |
| E2D2    | 28 enc / 21 dec |   1 |                         16.68 |
| E2D2    | 28 enc / 21 dec |   2 |                         32.83 |
| E2D2    | 28 enc / 21 dec |   4 |                         43.29 |
| BD3LM   | 28              |   1 |             